# Build Index kNN


### load data we have so far
- we can only index with integers, it would be great to add the path but we can't



In [5]:
import os
import pandas as pd

base_dir = "../src/embeddings/"

batches = list()
for _file in os.listdir(base_dir):
    if os.path.basename(_file).split(".")[-1] == "parquet":
        batches.append(pd.read_parquet(os.path.join(base_dir, _file)))
        
df = (
    pd.concat(batches)
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index":"contract_id"})
)
print(df.shape)
del df

(820000, 3)


## Annoy Library
there are 2 tradeoffs for building the index (from the docs)

- n_trees is provided during build time and affects the build time and the index size. A larger value will give more accurate results, but larger indexes.
- search_k is provided in runtime and affects the search performance. A larger value will give more accurate results, but will take longer time to return.


In [30]:
from annoy import AnnoyIndex
t = AnnoyIndex(150, 'dot')
t.set_seed(42)

### add items to index

In [31]:
%%time
for ix, row in df.iterrows():
    t.add_item(ix, row.loc["embedding"])

CPU times: user 4.45 s, sys: 23.2 ms, total: 4.48 s
Wall time: 4.48 s


In [32]:
n_batches = int(1083325 / 20000)
print(f"estimated time for loading all contracts is: {n_batches*4.48/60} minutes")

estimated time for loading all contracts is: 4.032 minutes


### build the index
- this is one of the parameters to tune, let's do low multithreading so i don't disturbe the embedding computation
- the variables to watch is memory and build time

In [33]:
%%time
t.build(10, n_jobs=3)
t.save('test.ann')

CPU times: user 6.55 s, sys: 147 ms, total: 6.69 s
Wall time: 2.74 s


True

In [34]:
! du -ah | grep test.ann

 56M	./test.ann


- ok that was very fast and not that much memory for the "model"
- let's check out how inference works

In [51]:
%%time
source_contract, embedding = df.loc[12,"contract_path"], df.loc[12,"embedding"]
res = t.get_nns_by_vector(embedding, 5, search_k=20, include_distances=False)
print(source_contract)
df.loc[res, "contract_path"].to_dict()

/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/7c/7CfFEEbF5d4Bcc701003ebd623a4378b2E023Da4_UniswapTest.sol
CPU times: user 1.11 ms, sys: 529 µs, total: 1.64 ms
Wall time: 1.13 ms


{30780: '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/fe/fE520341C202D9170EDe56F4DB763b27F4dc03E4_COIN.sol',
 46450: '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/57/5705b1C4a0Abf33281bEbb090f617e772516C21a_DT.sol',
 66772: '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/b2/b2B811F2B0ca741b2C13Be1d57278AD321c4720c_NfToken.sol',
 47767: '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/6f/6fbbe7c2aad159ad9bb38e4ec2a4e7bbcfbac36e_SmartInvestorGuys.sol',
 46359: '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/57/57d1bc7480a656D030B482456CBAb65d6Cd71B3F_RakuCoin.sol'}

- again very fast, i have no idea if the recommendations are good though...was manually looking through them but it's hard to tell for me
- let's package it into a function and try different parameters for build sizes and times

In [103]:
import time

def build_index(df, n_trees=20, n_jobs=3, output_path=None):
    st = time.time()
    t = AnnoyIndex(150, 'dot')
    t.set_seed(42)
    for ix, row in df.iterrows():
        t.add_item(ix, row.loc["embedding"])
    t.build(n_trees, n_jobs=n_jobs)
    
    if output_path is not None:
        t.save(output_path)
    elapsed = time.time() - st
    return t, elapsed

In [62]:
indexes = dict()
tree_dimensions = (10, 20, 50, 100, 200)
for n_trees in tree_dimensions:
    t, elapsed = build_index(df, n_trees=n_trees, n_jobs=3, output_path="test_{}.ann".format(n_trees))
    indexes[n_trees] = t
    print(f"build time for {n_trees} trees was {elapsed} seconds")

build time for 10 trees was 7.638796091079712 seconds
build time for 20 trees was 9.878449201583862 seconds
build time for 50 trees was 16.51998209953308 seconds
build time for 100 trees was 29.181003093719482 seconds
build time for 200 trees was 53.388516902923584 seconds


In [55]:
! du -ah | grep test_ | sort

 56M	./test_10.ann
 66M	./test_20.ann
 95M	./test_50.ann
142M	./test_100.ann
237M	./test_200.ann


In [57]:
from itertools import product

def inference(t, embedding, search_k=20):
    st = time.time()
    res = t.get_nns_by_vector(embedding, 5, search_k=20, include_distances=False)
    similar_contracts = df.loc[res, "contract_path"].to_dict()
    elapsed = time.time() - st
    return similar_contracts, elapsed
    
embedding = df.loc[23,"embedding"]
search_ks = (10, 20, 50)
for search_k, (n_trees, _t) in product(search_ks, indexes.items()):
    similar_contracts, elapsed = inference(_t, embedding, search_k=search_k)
    print(f"n_trees: {n_trees}, search_k:{search_k} -> inference: {elapsed} seconds")

n_trees: 10, search_k:10 -> inference: 0.020502805709838867 seconds
n_trees: 20, search_k:10 -> inference: 0.01873779296875 seconds
n_trees: 50, search_k:10 -> inference: 0.020534753799438477 seconds
n_trees: 100, search_k:10 -> inference: 0.027746915817260742 seconds
n_trees: 200, search_k:10 -> inference: 0.026354074478149414 seconds
n_trees: 10, search_k:20 -> inference: 0.00036716461181640625 seconds
n_trees: 20, search_k:20 -> inference: 0.0003590583801269531 seconds
n_trees: 50, search_k:20 -> inference: 0.0003719329833984375 seconds
n_trees: 100, search_k:20 -> inference: 0.0003972053527832031 seconds
n_trees: 200, search_k:20 -> inference: 0.0004780292510986328 seconds
n_trees: 10, search_k:50 -> inference: 0.0003330707550048828 seconds
n_trees: 20, search_k:50 -> inference: 0.0003440380096435547 seconds
n_trees: 50, search_k:50 -> inference: 0.0003719329833984375 seconds
n_trees: 100, search_k:50 -> inference: 0.00039076805114746094 seconds
n_trees: 200, search_k:50 -> inferen

- build time seems to scale relatively linearly
- memory footprint also scales pretty linearly
- inference time is also linear 
- all of them seem very low so we can try with a full dataset. the only concerning number is the memory footprint

In [96]:
! rm *.ann

# consolidate
- let's gather all the necessary functions into a file
- figure out what's the maximum memory footprint we want to have
- the not ideal part is we also need to have the dataframe loaded to get the contract_path to be able to return that. we can drop the embeddings at that point to save memory

In [92]:
import os
import time
from datetime import timedelta
from typing import Union

import numpy as np
import pandas as pd

from annoy import AnnoyIndex

base_dir = "embeddings/"

def get_batch_names(base_dir : str) -> str:
    batches = list()
    for _file in os.listdir(base_dir):
        if os.path.basename(_file).split(".")[-1] == "parquet":
            yield os.path.join(base_dir, _file)
            
def generate_dataset(base_dir : str) -> pd.DataFrame:  
    return (
        pd.concat([pd.read_parquet(batch_file) for batch_file in get_batch_names(base_dir)])
        .reset_index(drop=True)
    )

def build_index(df, n_trees=20, n_jobs=3, output_path=None):
    st = time.time()
    t = AnnoyIndex(150, 'dot')
    t.set_seed(42)
    for ix, row in df.iterrows():
        t.add_item(ix, row.loc["embedding"])
    t.build(n_trees, n_jobs=n_jobs)
    
    if output_path is not None:
        t.save(output_path)
    elapsed = time.time() - st
    return t, elapsed


def inference(t, embedding, search_k=20):
    st = time.time()
    res = t.get_nns_by_vector(embedding, 5, search_k=20, include_distances=False)
    similar_contracts = df.loc[res, "contract_path"].to_dict()
    elapsed = time.time() - st
    return similar_contracts, elapsed


class IndexBuilder(object):
    
    def __init__(self, pretrained=None, dataset_dir=None):
        # embedding size is fixed
        self._index = AnnoyIndex(150, 'dot')
        
        if pretrained is not None:
            self.load_pretrained(pretrained)
            
        if dataset_dir is not None:
            self._data = self.generate_dataset(dataset_dir)
    
    def load_pretrained(self, pretrained):
        self._index.load(pretrained)

    def get_batch_names(self, base_dir : str) -> str:
        batches = list()
        for _file in os.listdir(base_dir):
            if os.path.basename(_file).split(".")[-1] == "parquet":
                yield os.path.join(base_dir, _file)

    def generate_dataset(self, dataset_dir : str) -> pd.DataFrame:  
        return (
            pd.concat([pd.read_parquet(batch_file) for batch_file in get_batch_names(dataset_dir)])
            .reset_index(drop=True)
        )

    def build_index(self, n_trees=20, n_jobs=3, output_path=None):
        assert hasattr(IB, "_data"), "must load data before building index through dataset_dir"
        st = time.time()
        self._index.set_seed(42)
        for ix, row in self._data.iterrows():
            self._index.add_item(ix, row.loc["embedding"])
        self._index.build(n_trees, n_jobs=n_jobs)

        if output_path is not None:
            self._index.save(output_path)
        elapsed = time.time() - st
        print(f"time elapsed to build index: {timedelta(seconds=elapsed)}")
        
        # we can throw away the embeddings to save memory
        self._data = self._data["contract_path"].to_dict()
        
    def preprocess_contract(self, contract: Union[str, np.array]) -> np.array:
        if isinstance(contract, str):
            pass
        else:
            return contract
        
    def __call__(self, contract, k=5, search_k=20, include_distances=False):
        embedding = self.preprocess_contract(contract)
        res = self._index.get_nns_by_vector(embedding, k, search_k=20, include_distances=include_distances)
        similar_contracts = [self._data[_res] for _res in res]
        return similar_contracts


In [93]:
%%time
IB = IndexBuilder(dataset_dir="embeddings/")
IB.build_index(output_path="test20.ann")

time elapsed to build index: 0:00:09.295983
CPU times: user 18 s, sys: 321 ms, total: 18.3 s
Wall time: 9.64 s


In [94]:
similar_contracts = IB(embedding)
similar_contracts

['/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/ca/CAac6F5574d1C959D6B4Ff5c89344bee888fd89E_Trade.sol',
 '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/d1/d16Ba886292f20c29d5c31F9756cf40827da40a0_CopygameAxelarBridge.sol',
 '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/a4/A4501782232DC056EC4B358cAACbcDEF274f52Ab_NFT.sol',
 '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/b5/b52aB99F349F400941311531D9508AE63A4cf218_SkaleDKG.sol',
 '/Users/pablo/Documents/Coding/company_challenges/OpenZeppelin/smart-contract-sanctuary-ethereum/contracts/ropsten/e3/E3023D7A9984051035191D34c8B149f76F6FE8FE_BondPoolLibV1.sol']